In [ ]:
import pandas as pd
import geopandas as gpd
import polars as pl # Para cargar DENUE si es muy grande
import numpy as np
import json # Para cargar el diccionario de categorías SCIAN
from shapely.geometry import Point
from tqdm.auto import tqdm 
tqdm.pandas()


In [ ]:
# --- Rutas y Nombres de Archivo (Configurables) ---
RUTA_TIENDAS_CON_CENSO_INPUT = './datos/output/tiendas_train_final_con_censo_ageb.gpkg'
LAYER_TIENDAS_CON_CENSO = 'tiendas_censo_ageb_nl_tam' # Capa dentro del gpkg

# Archivos del DENUE (ajusta si es uno nacional o varios por estado)
RUTA_DENUE_RAW = './datos/denue_raw/' 
ARCHIVOS_DENUE_CSV = [
    RUTA_DENUE_RAW + 'denue_nuevo_leon.csv', # Ejemplo
    RUTA_DENUE_RAW + 'denue_tamaulipas.csv'  # Ejemplo
]
# Si es un solo archivo nacional:
# ARCHIVOS_DENUE_CSV = [RUTA_DENUE_RAW + 'denue_nacional_completo.csv'] 

# Nombres de columnas en tus archivos DENUE (¡VERIFICA ESTOS!)
COL_DENUE_LATITUD = 'latitud'
COL_DENUE_LONGITUD = 'longitud'
COL_DENUE_SCIAN = 'codigo_act' # o 'id_actividad', etc.
COL_DENUE_ENTIDAD_PARA_FILTRAR = 'entidad' # Si descargas nacional y necesitas filtrar por CVE_ENT de NL (19) y TAM (28)

# Archivo JSON con las categorías PDI y códigos SCIAN
RUTA_CATEGORIAS_SCIAN_JSON = './datos/output/categorias_pdi_scian_final.json'

# CRS de las tiendas (EPSG:6372, ya proyectado) y CRS original de DENUE (WGS84)
CRS_OBJETIVO = 'EPSG:6372'
CRS_DENUE_ORIGINAL = 'EPSG:4326' 

# Radios para análisis de conteo (en metros)
RADIOS_ANALISIS_M = [200, 500, 1000] 

# Archivo de salida
RUTA_DATOS_OUTPUT = './datos/output/'
ARCHIVO_SALIDA_GPKG = RUTA_DATOS_OUTPUT + 'tiendas_train_censo_denue.gpkg'
# --- Fin de Configuración ---

print("--- Iniciando Integración de Datos del DENUE ---")

In [ ]:
# --- 1. Cargar el GeoDataFrame de Tiendas (ya con Censo) ---
try:
    tiendas_gdf = gpd.read_file(RUTA_TIENDAS_CON_CENSO_INPUT, layer=LAYER_TIENDAS_CON_CENSO)
    print(f"GeoDataFrame de tiendas con censo cargado: {len(tiendas_gdf)} tiendas.")
    # Asegurar que el CRS es el correcto, si no, reproyectar.
    if tiendas_gdf.crs is None or tiendas_gdf.crs.to_string().upper() != CRS_OBJETIVO.upper():
        print(f"CRS de tiendas_gdf ({tiendas_gdf.crs}) no es el esperado ({CRS_OBJETIVO}). Reproyectando...")
        tiendas_gdf = tiendas_gdf.to_crs(CRS_OBJETIVO)
    print(f"CRS de tiendas_gdf: {tiendas_gdf.crs}")
except Exception as e:
    print(f"Error al cargar '{RUTA_TIENDAS_CON_CENSO_INPUT}': {e}")
    raise

In [ ]:
# --- 2. Cargar el Diccionario de Categorías PDI SCIAN ---
try:
    with open(RUTA_CATEGORIAS_SCIAN_JSON, 'r', encoding='utf-8') as f:
        categorias_pdi_scian = json.load(f)
    print(f"\nDiccionario de categorías PDI SCIAN cargado desde '{RUTA_CATEGORIAS_SCIAN_JSON}'.")
    # print(categorias_pdi_scian)
except FileNotFoundError:
    print(f"Error: Archivo JSON '{RUTA_CATEGORIAS_SCIAN_JSON}' no encontrado.")
    print("Asegúrate de haber ejecutado el notebook '00_Procesar_Catalogo_SCIAN_Avanzado.ipynb' y guardado el JSON.")
    raise
except Exception as e:
    print(f"Error al cargar el archivo JSON de categorías SCIAN: {e}")
    raise

In [ ]:
# --- 3. Cargar y Preparar Datos del DENUE ---
denue_dfs_list = []
print("\nCargando datos del DENUE...")
for archivo_csv in ARCHIVOS_DENUE_CSV:
    try:
        # Para archivos DENUE muy grandes, scan_csv de Polars es preferible
        # Aquí usamos Pandas por simplicidad, asumiendo que los archivos por estado son manejables
        # O que el filtrado por estado se hace primero si es un archivo nacional muy grande.
        df_temp = pd.read_csv(archivo_csv, low_memory=False, encoding='latin1') # DENUE suele venir en latin1
        # Si es un archivo nacional, filtrar por Nuevo León (19) y Tamaulipas (28)
        # if 'nombre_columna_clave_entidad_denue' in df_temp.columns: # Reemplaza con el nombre real
        #     df_temp = df_temp[df_temp['nombre_columna_clave_entidad_denue'].isin([19, 28])]
        denue_dfs_list.append(df_temp)
        print(f"  Cargadas {len(df_temp)} unidades de {archivo_csv} (después de posible filtro estatal).")
    except FileNotFoundError:
        print(f"  Error: Archivo DENUE no encontrado en '{archivo_csv}'. Omitiendo.")
    except Exception as e:
        print(f"  Error al cargar '{archivo_csv}': {e}. Omitiendo.")

if not denue_dfs_list:
    raise FileNotFoundError("No se pudo cargar ningún archivo del DENUE. Verifica rutas y nombres.")

denue_df_completo = pd.concat(denue_dfs_list, ignore_index=True)
print(f"Total de unidades DENUE cargadas (NL y TAM): {len(denue_df_completo)}")

# Verificar columnas y limpiar coordenadas
denue_cols_necesarias = [COL_DENUE_LATITUD, COL_DENUE_LONGITUD, COL_DENUE_SCIAN]
if not all(col in denue_df_completo.columns for col in denue_cols_necesarias):
    print(f"Error: Faltan columnas esenciales en DENUE. Necesarias: {denue_cols_necesarias}")
    print(f"Columnas disponibles: {denue_df_completo.columns.tolist()}")
    raise ValueError("Columnas faltantes en DENUE.")

denue_df_completo[COL_DENUE_LATITUD] = pd.to_numeric(denue_df_completo[COL_DENUE_LATITUD], errors='coerce')
denue_df_completo[COL_DENUE_LONGITUD] = pd.to_numeric(denue_df_completo[COL_DENUE_LONGITUD], errors='coerce')
denue_df_completo.dropna(subset=[COL_DENUE_LATITUD, COL_DENUE_LONGITUD], inplace=True)
denue_df_completo[COL_DENUE_SCIAN] = denue_df_completo[COL_DENUE_SCIAN].astype(str).str.strip() # Estandarizar SCIAN a string

print(f"Unidades DENUE después de limpiar coordenadas: {len(denue_df_completo)}")

# Convertir DENUE a GeoDataFrame y reproyectar
try:
    denue_gdf = gpd.GeoDataFrame(
        denue_df_completo,
        geometry=gpd.points_from_xy(denue_df_completo[COL_DENUE_LONGITUD], denue_df_completo[COL_DENUE_LATITUD]),
        crs=CRS_DENUE_ORIGINAL
    )
    if denue_gdf.crs != tiendas_gdf.crs:
        print(f"Reproyectando DENUE GDF de {denue_gdf.crs} a {tiendas_gdf.crs}...")
        denue_gdf = denue_gdf.to_crs(tiendas_gdf.crs)
    print(f"GeoDataFrame del DENUE preparado. CRS: {denue_gdf.crs}")
except Exception as e:
    print(f"Error al crear o reproyectar GeoDataFrame del DENUE: {e}")
    raise

In [ ]:
# --- 4. Filtrar DENUE por Categorías de PDI y Generar Características ---
# Usaremos el diccionario `categorias_pdi_scian` cargado del JSON

gdfs_pdi_denue_filtrados = {}
print("\nFiltrando DENUE por categorías PDI definidas...")
for categoria, scian_codes_list in categorias_pdi_scian.items():
    if not scian_codes_list: # Si la lista de códigos está vacía para una categoría
        print(f"  Categoría '{categoria}' no tiene códigos SCIAN definidos. Omitiendo.")
        gdfs_pdi_denue_filtrados[categoria] = gpd.GeoDataFrame(columns=denue_gdf.columns, geometry=[], crs=denue_gdf.crs) # GDF vacío
        continue
    
    # Asegurar que los códigos SCIAN a filtrar sean strings
    scian_codes_str = [str(code).strip() for code in scian_codes_list]
    
    # Filtrar usando isin para códigos exactos de 6 dígitos
    # Si quieres usar prefijos (ej. '611' para todo educación), usarías .str.startswith()
    temp_gdf = denue_gdf[denue_gdf[COL_DENUE_SCIAN].isin(scian_codes_str)]
    gdfs_pdi_denue_filtrados[categoria] = temp_gdf
    print(f"  Categoría '{categoria}': {len(temp_gdf)} establecimientos encontrados.")

In [ ]:
# -- 4.A Contar PDIs en Radios ---
print("\nCalculando conteo de PDIs del DENUE en radios alrededor de cada tienda OXXO...")
for radio_m in RADIOS_ANALISIS_M:
    print(f"  Procesando para radio de {radio_m} metros...")
    # Crear buffer alrededor de las tiendas OXXO una sola vez por radio
    tiendas_buffer_gdf = tiendas_gdf.copy() # Trabajar sobre una copia para los buffers
    tiendas_buffer_gdf['geometry_buffer'] = tiendas_buffer_gdf.geometry.buffer(radio_m)
    # Usar la geometría del buffer para el sjoin
    tiendas_buffer_gdf = tiendas_buffer_gdf.set_geometry('geometry_buffer')


    for categoria, pdi_gdf_cat in tqdm(gdfs_pdi_denue_filtrados.items(), desc=f"Cat. DENUE (radio {radio_m}m)"):
        nombre_col_conteo = f'denue_conteo_{categoria}_{radio_m}m'
        if not pdi_gdf_cat.empty:
            # Unión espacial para identificar qué PDIs caen en qué buffers de tienda
            # sjoin_predicates=['intersects'] es el nuevo nombre para 'op' en versiones recientes de geopandas
            # o simplemente op='intersects' para versiones más antiguas
            try: # Intentar con la sintaxis más nueva primero
                 joined_conteo = gpd.sjoin(tiendas_buffer_gdf[['TIENDA_ID', 'geometry_buffer']], 
                                      pdi_gdf_cat[['geometry']], 
                                      how='left', 
                                      predicate='intersects') # o 'contains' si el PDI debe estar completamente dentro
            except TypeError: # Fallback a la sintaxis antigua
                 joined_conteo = gpd.sjoin(tiendas_buffer_gdf[['TIENDA_ID', 'geometry_buffer']], 
                                      pdi_gdf_cat[['geometry']], 
                                      how='left', 
                                      op='intersects')

            # Contar PDIs por TIENDA_ID
            conteo_por_tienda = joined_conteo.dropna(subset=['index_right']).groupby('TIENDA_ID').size()
            tiendas_gdf[nombre_col_conteo] = tiendas_gdf['TIENDA_ID'].map(conteo_por_tienda).fillna(0).astype(int)
        else:
            tiendas_gdf[nombre_col_conteo] = 0
        # print(f"    Columna creada: {nombre_col_conteo}, suma: {tiendas_gdf[nombre_col_conteo].sum()}")

In [ ]:
# -- 4.B Distancia al PDI más Cercano ---
# NOTA: Este bucle puede ser MUY LENTO. Para un datathon, considera:
# 1. Hacerlo solo para las 1-2 categorías MÁS IMPORTANTES (ej. competidores_directos).
# 2. Si el tiempo apremia, omitir este paso o hacerlo sobre una muestra.
# 3. Investigar/implementar optimizaciones con rtree o cKDTree si tienes mucha experiencia y tiempo.

print("\nCalculando distancia al PDI del DENUE más cercano por categoría (puede tardar)...")
for categoria, pdi_gdf_cat in tqdm(gdfs_pdi_denue_filtrados.items(), desc="Cat. DENUE (distancia)"):
    nombre_col_dist = f'denue_dist_{categoria}_cercano_m'
    if not pdi_gdf_cat.empty and not pdi_gdf_cat.geometry.is_empty.all() and len(pdi_gdf_cat) > 0:
        # Crear un árbol espacial para los PDIs de esta categoría para búsquedas eficientes
        # Esto requiere la biblioteca 'rtree' instalada: uv pip install rtree
        try:
            sindex_pdi_cat = pdi_gdf_cat.sindex
            
            distancias = []
            for idx_tienda, tienda_row in tqdm(tiendas_gdf.iterrows(), total=len(tiendas_gdf), desc=f"  Dist. {categoria}", leave=False):
                tienda_geom = tienda_row.geometry
                # Obtener posibles candidatos cercanos usando el índice espacial
                posibles_matches_indices = list(sindex_pdi_cat.intersection(tienda_geom.buffer(max(RADIOS_ANALISIS_M) * 2).bounds)) # Buscar en un radio amplio
                
                if not posibles_matches_indices:
                    distancias.append(np.nan) # O un valor grande como 99999
                    continue
                
                candidatos_cercanos_gdf = pdi_gdf_cat.iloc[posibles_matches_indices]
                
                if candidatos_cercanos_gdf.empty:
                     distancias.append(np.nan)
                     continue

                # Calcular distancias solo a estos candidatos
                min_dist = candidatos_cercanos_gdf.geometry.distance(tienda_geom).min()
                distancias.append(min_dist)

            tiendas_gdf[nombre_col_dist] = distancias
            tiendas_gdf[nombre_col_dist] = tiendas_gdf[nombre_col_dist].fillna(99999).astype(float) # Imputar NaNs y asegurar float
        except Exception as e_dist_optim: # Fallback a método simple si sindex falla o rtree no está
            print(f"    Advertencia: Falló el cálculo optimizado de distancia para {categoria} ({e_dist_optim}). Usando método simple (más lento)...")
            distancias_simple = tiendas_gdf.geometry.progress_apply(lambda g: pdi_gdf_cat.geometry.distance(g).min() if not pdi_gdf_cat.empty else np.nan)
            tiendas_gdf[nombre_col_dist] = distancias_simple.fillna(99999).astype(float)
            
    else: # Si no hay PDIs en la categoría
        tiendas_gdf[nombre_col_dist] = 99999.0 
    
    mean_dist_val = tiendas_gdf[nombre_col_dist][tiendas_gdf[nombre_col_dist] < 99999].mean()
    print(f"  Columna creada: {nombre_col_dist}, promedio dist (excl. 99999): {mean_dist_val:.2f} m")

In [ ]:
# --- 5. Guardar el Resultado ---
print("\n--- Guardando GeoDataFrame Enriquecido con DENUE ---")
# Asegurar que la columna de geometría original de las tiendas se mantenga si es necesario
# El sjoin de conteo con buffer podría haberla modificado si no se manejó con cuidado
# tiendas_gdf = tiendas_gdf.set_geometry('geometry') # Asegura que 'geometry' es la columna activa

print("Primeras filas del dataset final con datos DENUE:")
pd.set_option('display.max_columns', None)
print(tiendas_gdf.head())
print("\nColumnas finales:")
print(tiendas_gdf.columns.tolist())

try:
    tiendas_gdf.to_file(ARCHIVO_SALIDA_GPKG, driver='GPKG', layer='tiendas_censo_denue')
    print(f"\nDataset enriquecido guardado como GeoPackage en '{ARCHIVO_SALIDA_GPKG}'")
except Exception as e:
    print(f"Error al guardar el archivo de salida GeoPackage: {e}")

print("\n--- Integración de Datos del DENUE Completada ---")